In [46]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import matplotlib
%matplotlib inline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
import opendatasets as od
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor, plot_tree, export_text
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error
from datetime import datetime
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from joblib import dump

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10,6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [2]:
ross_df = pd.read_csv('train.csv', low_memory=False)
store_df = pd.read_csv('store.csv')
test_df = pd.read_csv('test.csv')
submission_df = pd.read_csv('sample_submission.csv')

In [3]:
store_df

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1110,1111,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1111,1112,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
1112,1113,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
1113,1114,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN


In [4]:
merged_df = ross_df.merge(store_df, how='left', on='Store')
merged_test_df = test_df.merge(store_df, how='left', on='Store')
merged_df.columns, 

(Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
        'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
        'CompetitionDistance', 'CompetitionOpenSinceMonth',
        'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
        'Promo2SinceYear', 'PromoInterval'],
       dtype='object'),)

In [5]:
merged_df.corr(numeric_only=True)['Sales'].sort_values(ascending=False)

Sales                        1.000000
Customers                    0.894711
Open                         0.678472
Promo                        0.452345
SchoolHoliday                0.085124
Promo2SinceWeek              0.059558
CompetitionOpenSinceYear     0.012659
Store                        0.005126
CompetitionDistance         -0.019229
Promo2SinceYear             -0.021127
CompetitionOpenSinceMonth   -0.028257
Promo2                      -0.091040
DayOfWeek                   -0.462125
Name: Sales, dtype: float64

In [38]:
merged_df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,Day,WeekOfYear,CompetitionOpen,Promo2Open,IsPromo2Month
0,1,5,2015-07-31,5263,555,1,1,0,1,c,...,NaN,NaN,NaN,2015,7,31,31,82.0,0.000000,0
1,2,5,2015-07-31,6064,625,1,1,0,1,a,...,13.0,2010.0,"Jan,Apr,Jul,Oct",2015,7,31,31,92.0,64.131148,1
2,3,5,2015-07-31,8314,821,1,1,0,1,a,...,14.0,2011.0,"Jan,Apr,Jul,Oct",2015,7,31,31,103.0,51.901639,1
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,...,NaN,NaN,NaN,2015,7,31,31,70.0,0.000000,0
4,5,5,2015-07-31,4822,559,1,1,0,1,a,...,NaN,NaN,NaN,2015,7,31,31,3.0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016776,682,2,2013-01-01,3375,566,1,0,a,1,b,...,NaN,NaN,NaN,2013,1,1,1,76.0,0.000000,0
1016827,733,2,2013-01-01,10765,2377,1,0,a,1,b,...,NaN,NaN,NaN,2013,1,1,1,159.0,0.000000,0
1016863,769,2,2013-01-01,5035,1248,1,0,a,1,b,...,48.0,2012.0,"Jan,Apr,Jul,Oct",2013,1,1,1,0.0,1.213115,1
1017042,948,2,2013-01-01,4491,1039,1,0,a,1,b,...,NaN,NaN,NaN,2013,1,1,1,0.0,0.000000,0


In [7]:
def split_date(df):
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['Year']= df.Date.dt.year
    df['Month'] = df.Date.dt.month
    df['Day'] = df.Date.dt.day
    df['WeekOfYear'] = df.Date.dt.isocalendar().week

In [8]:
split_date(merged_df)
split_date(merged_test_df)

In [9]:
def comp_month(df):
    df['CompetitionOpen'] = 12 * (df.Year - df.CompetitionOpenSinceYear) + (df.Month - df.CompetitionOpenSinceMonth)
    df['CompetitionOpen'] = df['CompetitionOpen'].map(lambda x : 0 if x < 0 else x).fillna(0)

In [10]:
comp_month(merged_df)
comp_month(merged_test_df)

In [11]:
merged_df[['Date',  'CompetitionDistance', 'CompetitionOpenSinceYear', 'CompetitionOpenSinceMonth', 'CompetitionOpen']]

,Date,CompetitionDistance,CompetitionOpenSinceYear,CompetitionOpenSinceMonth,CompetitionOpen
0,2015-07-31,1270.0,2008.0,9.0,82.0
1,2015-07-31,570.0,2007.0,11.0,92.0
2,2015-07-31,14130.0,2006.0,12.0,103.0
3,2015-07-31,620.0,2009.0,9.0,70.0
4,2015-07-31,29910.0,2015.0,4.0,3.0
...,...,...,...,...,...
1017204,2013-01-01,1900.0,2014.0,6.0,0.0
1017205,2013-01-01,1880.0,2006.0,4.0,81.0
1017206,2013-01-01,9260.0,NaN,NaN,0.0
1017207,2013-01-01,870.0,NaN,NaN,0.0


In [12]:
def check_promo_month(row):
    month2str = {
        1:'Jan',
        2:'Feb',
        3:'Mar',
        4:'Apr',
        5:'May',
        6:'Jun',
        7:'Jul',
        8:'Aug',
        9:'Sept',
        10:'Oct',
        11:'Nov',
        12:'Dec'
    }
    try:
        months = (row['PromoInterval'] or '').split(',')
        if row['Promo2Open'] and month2str[row['Month']] in months:
            return 1
        else:
            return 0
    except Exception:
        return 0
    

def promo_cols(df):
    df['Promo2Open'] = 12 * (df.Year - df.Promo2SinceYear) + (df.WeekOfYear - df.Promo2SinceWeek) * 7/30.5
    df['Promo2Open'] = df['Promo2Open'].map(lambda x : 0 if x < 0 else x).fillna(0) * df['Promo2']
    df['IsPromo2Month'] = df.apply(check_promo_month, axis=1) * df['Promo2']

In [13]:
promo_cols(merged_df)
promo_cols(merged_test_df)

In [14]:
merged_df[['Date', 'Promo2', 'Promo2SinceYear', 'Promo2SinceWeek', 'PromoInterval', 'Promo2Open', 'IsPromo2Month']]

,Date,Promo2,Promo2SinceYear,Promo2SinceWeek,PromoInterval,Promo2Open,IsPromo2Month
0,2015-07-31,0,NaN,NaN,NaN,0.000000,0
1,2015-07-31,1,2010.0,13.0,"Jan,Apr,Jul,Oct",64.131148,1
2,2015-07-31,1,2011.0,14.0,"Jan,Apr,Jul,Oct",51.901639,1
3,2015-07-31,0,NaN,NaN,NaN,0.000000,0
4,2015-07-31,0,NaN,NaN,NaN,0.000000,0
...,...,...,...,...,...,...,...
1017204,2013-01-01,1,2013.0,31.0,"Jan,Apr,Jul,Oct",0.000000,0
1017205,2013-01-01,0,NaN,NaN,NaN,0.000000,0
1017206,2013-01-01,0,NaN,NaN,NaN,0.000000,0
1017207,2013-01-01,0,NaN,NaN,NaN,0.000000,0


In [15]:
merged_df = merged_df[merged_df['Open'] == 1].copy()

In [16]:
input_cols = ['Store', 'DayOfWeek', 'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpen', 'Day', 'Month', 'Year', 'WeekOfYear', 'Promo2', 'Promo2Open', 'IsPromo2Month']
target_col = 'Sales'

In [17]:
inputs = merged_df[input_cols].copy()
target = merged_df[target_col].copy()
test_inputs = merged_test_df[input_cols].copy()

In [18]:
numeric_cols = ['Store','Promo', 'SchoolHoliday', 'CompetitionDistance', 'CompetitionOpen', 'Promo2', 'Promo2Open', 'IsPromo2Month', 'Day','Month', 'Year', 'WeekOfYear']
categorical_cols = ['Day', 'StateHoliday', 'StoreType', 'Assortment']

In [19]:
inputs[numeric_cols].isna().sum()

Store                     0
Promo                     0
SchoolHoliday             0
CompetitionDistance    2186
CompetitionOpen           0
Promo2                    0
Promo2Open                0
IsPromo2Month             0
Day                       0
Month                     0
Year                      0
WeekOfYear                0
dtype: int64

In [20]:
test_inputs[numeric_cols].isna().sum()

Store                   0
Promo                   0
SchoolHoliday           0
CompetitionDistance    96
CompetitionOpen         0
Promo2                  0
Promo2Open              0
IsPromo2Month           0
Day                     0
Month                   0
Year                    0
WeekOfYear              0
dtype: int64

In [21]:
inputs[categorical_cols].isna().sum()

Day             0
StateHoliday    0
StoreType       0
Assortment      0
dtype: int64

In [22]:
test_inputs[categorical_cols].isna().sum()

Day             0
StateHoliday    0
StoreType       0
Assortment      0
dtype: int64

In [23]:
max_dist = inputs['CompetitionDistance'].max()
max_dist

np.float64(75860.0)

In [24]:
inputs['CompetitionDistance'] = inputs['CompetitionDistance'].fillna(max_dist*2)
test_inputs['CompetitionDistance'] = test_inputs['CompetitionDistance'].fillna(max_dist*2)

In [25]:
scaler = MinMaxScaler().fit(inputs[numeric_cols])

In [26]:
inputs[numeric_cols] = scaler.transform(inputs[numeric_cols])
test_inputs[numeric_cols] = scaler.transform(test_inputs[numeric_cols])

In [27]:
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit(inputs[categorical_cols])
encoded_cols = list(encoder.get_feature_names_out(categorical_cols))

In [28]:
inputs[encoded_cols] = encoder.transform(inputs[categorical_cols])
test_inputs[encoded_cols] = encoder.transform(test_inputs[categorical_cols])

In [29]:
x_train = inputs[numeric_cols+encoded_cols]
x_test = test_inputs[numeric_cols+encoded_cols]

In [30]:
x_train.columns

Index(['Store', 'Promo', 'SchoolHoliday', 'CompetitionDistance',
       'CompetitionOpen', 'Promo2', 'Promo2Open', 'IsPromo2Month', 'Day',
       'Month', 'Year', 'WeekOfYear', 'Day_0.0', 'Day_0.03333333333333333',
       'Day_0.06666666666666668', 'Day_0.1', 'Day_0.13333333333333333',
       'Day_0.16666666666666669', 'Day_0.2', 'Day_0.23333333333333334',
       'Day_0.26666666666666666', 'Day_0.3', 'Day_0.3333333333333333',
       'Day_0.3666666666666667', 'Day_0.4', 'Day_0.43333333333333335',
       'Day_0.4666666666666667', 'Day_0.5', 'Day_0.5333333333333333',
       'Day_0.5666666666666667', 'Day_0.6', 'Day_0.6333333333333333',
       'Day_0.6666666666666666', 'Day_0.7', 'Day_0.7333333333333333',
       'Day_0.7666666666666667', 'Day_0.8', 'Day_0.8333333333333334',
       'Day_0.8666666666666667', 'Day_0.9', 'Day_0.9333333333333333',
       'Day_0.9666666666666667', 'Day_0.9999999999999999', 'StateHoliday_0',
       'StateHoliday_a', 'StateHoliday_b', 'StateHoliday_c', 'StoreType_

In [37]:
store_static_dict = store_df.set_index('Store').to_dict(orient='index')
store_static_dict[1]

{'StoreType': 'c',
 'Assortment': 'a',
 'CompetitionDistance': 1270.0,
 'CompetitionOpenSinceMonth': 9.0,
 'CompetitionOpenSinceYear': 2008.0,
 'Promo2': 0,
 'Promo2SinceWeek': nan,
 'Promo2SinceYear': nan,
 'PromoInterval': nan}

In [31]:
#['Store', 'Date', 'Promo', 'StateHoliday', 'SchoolHoliday']

In [51]:
print(x_train.columns.tolist())


['Store', 'Promo', 'SchoolHoliday', 'CompetitionDistance', 'CompetitionOpen', 'Promo2', 'Promo2Open', 'IsPromo2Month', 'Day', 'Month', 'Year', 'WeekOfYear', 'Day_0.0', 'Day_0.03333333333333333', 'Day_0.06666666666666668', 'Day_0.1', 'Day_0.13333333333333333', 'Day_0.16666666666666669', 'Day_0.2', 'Day_0.23333333333333334', 'Day_0.26666666666666666', 'Day_0.3', 'Day_0.3333333333333333', 'Day_0.3666666666666667', 'Day_0.4', 'Day_0.43333333333333335', 'Day_0.4666666666666667', 'Day_0.5', 'Day_0.5333333333333333', 'Day_0.5666666666666667', 'Day_0.6', 'Day_0.6333333333333333', 'Day_0.6666666666666666', 'Day_0.7', 'Day_0.7333333333333333', 'Day_0.7666666666666667', 'Day_0.8', 'Day_0.8333333333333334', 'Day_0.8666666666666667', 'Day_0.9', 'Day_0.9333333333333333', 'Day_0.9666666666666667', 'Day_0.9999999999999999', 'StateHoliday_0', 'StateHoliday_a', 'StateHoliday_b', 'StateHoliday_c', 'StoreType_a', 'StoreType_b', 'StoreType_c', 'StoreType_d', 'Assortment_a', 'Assortment_b', 'Assortment_c']


In [53]:
categorical_cols = ['StateHoliday', 'StoreType', 'Assortment']
numerical_cols = ['Promo', 'SchoolHoliday', 'CompetitionDistance',
                    'CompetitionOpen', 'Promo2', 'Promo2Open',
                    'IsPromo2Month', 'Day', 'Month', 'Year', 'WeekOfYear']
    
preprocessor = ColumnTransformer(transformers=[
        ('num', MinMaxScaler(), numeric_cols)
    ])
    
pipeline = Pipeline(steps=[
        ('preprocessing', preprocessor),
        ('model', XGBRegressor())
    ])
    
pipeline.fit(x_train, target)
dump(pipeline, 'xgb_pipeline.joblib')

['xgb_pipeline.joblib']

In [ ]:
def build_sample_input(store_id, date_str, promo, state_holiday, school_holiday, store_static_dict):
    
    # conveting given date by user into year, month, day, date, weekofyear
    date = pd.to_datetime(date_str)
    year = date.year
    month = date.month
    day = date.day
    WeekOfYear = date.isocalendar()[1]
    
    #storing data in dictionary
    store_data = store_static_dict[store_id]
    
    
    # calculating comp month and comp year from given data in store_df
    comp_month = 12 * (year - store_data.get('CompetitionOpenSinceYear', year)) + (month - store_data.get('CompetitionOpenSinceMonth', month))
    comp_month = max(0, comp_month) if not pd.isna(comp_month) else 0
    
    
    # calculating promo_to_open from week and promo year given data
    promo2_week = store_data.get('Promo2SinceWeek')
    promo2_year = store_data.get('Promo2SinceYear')
    
    if not pd.isna(promo2_week) and not pd.isna(promo2_year):
        promo2_open = 12 * (year - promo2_year) + ((WeekOfYear - promo2_week)* 7 / 30.5)
        promo2_open = max(0, promo2_open) * store_data.get('Promo2', 0)
    else:
        promo2_open = 0
        
    month2_str = {
        'Jan':1,
        'Feb':2,
        'Mar':3,
        'Apr':4,
        'May':5,
        'Jun':6,
        'Jul':7,
        'Aug':8,
        'Sept':9,
        'Oct':10,
        'Nov':11,
        'Dec':12
    }
    
    
    promo_interval = store_data.get('PromoInterval')
    if store_data.get('Promo2', 0 ) == 1 and isinstance(promo_interval,str):
        is_promo2_month = int(month2_str[month] in promo_interval.split(',') )
    else:
        is_promo2_month = 0    
    
    
    
    

In [42]:
x_train.columns

Index(['Store', 'Promo', 'SchoolHoliday', 'CompetitionDistance',
       'CompetitionOpen', 'Promo2', 'Promo2Open', 'IsPromo2Month', 'Day',
       'Month', 'Year', 'WeekOfYear', 'Day_0.0', 'Day_0.03333333333333333',
       'Day_0.06666666666666668', 'Day_0.1', 'Day_0.13333333333333333',
       'Day_0.16666666666666669', 'Day_0.2', 'Day_0.23333333333333334',
       'Day_0.26666666666666666', 'Day_0.3', 'Day_0.3333333333333333',
       'Day_0.3666666666666667', 'Day_0.4', 'Day_0.43333333333333335',
       'Day_0.4666666666666667', 'Day_0.5', 'Day_0.5333333333333333',
       'Day_0.5666666666666667', 'Day_0.6', 'Day_0.6333333333333333',
       'Day_0.6666666666666666', 'Day_0.7', 'Day_0.7333333333333333',
       'Day_0.7666666666666667', 'Day_0.8', 'Day_0.8333333333333334',
       'Day_0.8666666666666667', 'Day_0.9', 'Day_0.9333333333333333',
       'Day_0.9666666666666667', 'Day_0.9999999999999999', 'StateHoliday_0',
       'StateHoliday_a', 'StateHoliday_b', 'StateHoliday_c', 'StoreType_